In [1]:
import os
os.chdir('..')

In [2]:
import inspect
import multiprocessing as mp
from typing import List, Tuple

import numba as nb
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt

from neo_backtesting import Backtester, run_gridsearch
from strategy import boll_inbar
from util import read_candle_feather, transform_candle_np_struct, transform_np_struct
from joblib import Parallel, delayed

%load_ext line_profiler
%matplotlib inline

In [3]:
# 回测起始日
START_DATE = '20180301'
END_DATE = '20220614'

# 布林带周期与宽度
N = 100
B = 2

# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万
FACE_VALUE = 0.001  # 合约面值 0.001

COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 5

CONTRACT_TYPE = 'futures'  # 正向合约

# 模拟器参数
SIMULATOR_PARAMS = {
    'init_capital': INIT_CAPITAL, 
    'face_value': FACE_VALUE, 
    'comm_rate': COMM_RATE, 
    'liqui_rate': LIQUI_RATE, 
    'init_pos': 0
}

STRA = boll_inbar  # 要回测的策略

MIN_INTERVAL = '1m'  # 最小回测周期

ETH_PATHS = {
    '1m': '/home/lostleaf/feather_data/spot/ETH-USDT_1m.fea',
    '30m': '/home/lostleaf/feather_data/spot/ETH-USDT_30m.fea',
    '1h': '/home/lostleaf/feather_data/spot/ETH-USDT_1h.fea'
}

In [4]:
%%time
backtester = Backtester(
    candle_paths=ETH_PATHS, 
    contract_type='futures', 
    simulator_params=SIMULATOR_PARAMS, 
    stra_module=boll_inbar)
factor_params = {
    'n': 100,
    'b': 1.8,
    'itl': '1h'
}
strategy_params = {
    'leverage': 1.5
}

results = backtester.run_detailed(START_DATE, END_DATE, INIT_CAPITAL, FACE_VALUE, factor_params, strategy_params)
results['equity'] /= results['equity'].iat[0]
results

CPU times: user 2.94 s, sys: 1.48 s, total: 4.42 s
Wall time: 3.77 s


,candle_begin_time,candle_end_time,open,high,low,close,volume,upper,median,lower,pos,equity
275636,2018-03-01 00:00:00,2018-03-01 00:01:00,853.50,853.75,852.75,853.00,13.72510,894.846865,858.6274,822.407935,0,1.000000
275637,2018-03-01 00:01:00,2018-03-01 00:02:00,853.51,853.51,852.60,852.80,20.10843,894.846865,858.6274,822.407935,0,1.000000
275638,2018-03-01 00:02:00,2018-03-01 00:03:00,853.41,853.41,852.80,853.01,26.58684,894.846865,858.6274,822.407935,0,1.000000
275639,2018-03-01 00:03:00,2018-03-01 00:04:00,853.01,853.39,852.61,852.97,19.51853,894.846865,858.6274,822.407935,0,1.000000
275640,2018-03-01 00:04:00,2018-03-01 00:05:00,852.97,852.97,850.99,851.00,100.14852,894.846865,858.6274,822.407935,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2524870,2022-06-13 23:56:00,2022-06-13 23:57:00,1206.46,1209.86,1205.19,1207.06,727.10310,1866.223210,1534.1303,1202.037390,-52796956,888.699028
2524871,2022-06-13 23:57:00,2022-06-13 23:58:00,1207.07,1209.94,1205.86,1205.93,774.45520,1866.223210,1534.1303,1202.037390,-52796956,889.295634
2524872,2022-06-13 23:58:00,2022-06-13 23:59:00,1205.93,1207.62,1203.87,1206.07,550.73550,1866.223210,1534.1303,1202.037390,-52796956,889.221718
2524873,2022-06-13 23:59:00,2022-06-14 00:00:00,1206.07,1212.76,1205.03,1209.82,1152.11320,1862.409122,1528.3911,1194.373078,-52796956,887.241832


In [5]:
%%time

df = run_gridsearch(
    stra_module=STRA, 
    candle_paths=ETH_PATHS, 
    contract_type='futures', 
    simulator_params=SIMULATOR_PARAMS, 
    start_date=START_DATE, 
    end_date=END_DATE, 
    init_capital=INIT_CAPITAL, 
    face_value=FACE_VALUE, 
    n_proc=16)

df = df.sort_values('equity', ascending=False, ignore_index=True)
print(df.shape)
print(df.head())

(1980, 6)
        equity  itl    n    b  leverage  face_value
0  1439.566035  30m  415  2.0       1.5       0.001
1  1299.500540  30m  410  2.0       1.5       0.001
2  1291.816028   1h  200  2.0       1.5       0.001
3  1266.666791   1h  210  2.0       1.5       0.001
4  1234.383470   1h  205  2.0       1.5       0.001
CPU times: user 54.5 ms, sys: 295 ms, total: 350 ms
Wall time: 3min 39s
